# Probabilistic classification algorithms

In this tutorial we will implement a Naive Bayes classifier for text classification. More specifically, the aim of the classifier is to predict whether a given product review is positive or negative.

We will represent documents (i.e. product reviews) as vectors of binary features. For each [unigram or bigram](https://en.wikipedia.org/wiki/N-gram) that appears in at least one document in our training dataset we introduce a binary feature. For a given document a feature is 1 if the documents contains the corresponding unigram/bigram, and it is 0 otherwise.

The datasets are contained in four files ('test.negative', 'test.positive', 'train.negative', 'train.positive'). The files correspond to product reviews from the test and train datasets labelled positively or negatively. For example, the file 'test.negative' contains negative product reviews from the test dataset.

The reviews are preprocessed and expressed as a list of unigrams and bigrams without duplications (for a bigram (w1,w2), the corresponding record is w1__w2). Each review is represented as a single line and features (unigrams and bigrams) extracted from that review are listed space delimited. Take a moment to inspect the content of the dataset files.

In [2]:
import numpy as np

1. First, we read the training data, compute feature statistics, and store it in a [dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) $\texttt{featureStat}$, where $\texttt{featureStat[feature][classLabel]}$ is equal to the number of documents in the class $\texttt{classLabel}$ that contain $\texttt{feature}$. We also compute the total number of positive train instances and the total number of negative train instances.

In [3]:
def count(featureStat, fname, classLabel):
    numInstances = 0
    with open(fname) as file:
        for line in file:
            numInstances += 1
            for feature in line.strip().split():
                if feature not in featureStat:
                    featureStat[feature] = {0:0, 1:0}
                featureStat[feature][classLabel] += 1
    return numInstances

In [4]:
featureStat = {}
numPositiveInstances = count(featureStat, "Probabilistic classification algorithms/train.positive", 1)
numNegativeInstances = count(featureStat, "Probabilistic classification algorithms/train.negative", 0)

2. Now, compute the conditional probabilities $P(w_i = 1 | C = c)$, i.e. probability of specific feature $w_i$ to be present in a document from class $c$. Use Laplace smooting to avoid zero probabilities.

In [4]:
def compute_conditional_probabilities(featureStat, numPositiveInstances, numNegativeInstances):
    conditionalProb = {}

    for feature, counts in featureStat.items():
        conditionalProb[feature] = {
            1: (counts[1] + 1) / (numPositiveInstances + 2),
            0: (counts[0] + 1) / (numNegativeInstances + 2),
        }
    
    return conditionalProb

In [10]:
conditionalProb =  compute_conditional_probabilities(featureStat, numPositiveInstances, numNegativeInstances)

3. Implement a Naive Bayes classfier function that predicts whether a given document belongs to the positive or the negative class. To avoid problems of very small numbers, instead of computing $P(C=c) \cdot \prod_i P(w_i = a_i| C=c)$ cosider computing the $\log$ of that.

In [6]:
def predict_naive_bayes(document, conditionalProb, numPositiveInstances, numNegativeInstances):
    """Predicts the class (0: negative, 1: positive) of a given document."""
    
    # Prior probabilities with Laplace smoothing
    totalInstances = numPositiveInstances + numNegativeInstances
    logPriorPositive = np.log(numPositiveInstances / totalInstances + 2)
    logPriorNegative = np.log(numNegativeInstances / totalInstances)

    # Compute log probabilities
    logProbPositive = logPriorPositive
    logProbNegative = logPriorNegative

    for feature in conditionalProb:
        if feature in document:
            logProbPositive += np.log(conditionalProb[feature][1])
            logProbNegative += np.log(conditionalProb[feature][0])
        else:
            logProbPositive += np.log(1 - conditionalProb[feature][1])
            logProbNegative += np.log(1 - conditionalProb[feature][0])
    
    # Return the predicted class (1 for positive, 0 for negative)
    return 1 if logProbPositive > logProbNegative else 0


4. Let's now read the test dataset from the files

In [ ]:
def getInstances(fname, classLabel):
    data = []
    with open(fname) as file:
        for line in file:
            data.append((classLabel, line.strip().split()))
    return data

# Read test data
test_data = getInstances("Probabilistic classification algorithms/test.positive", 1)
test_data.extend(getInstances("Probabilistic classification algorithms/test.negative", 0))

4. Evaluate accuracy of the Naive Bayes algorithm on the test data

In [11]:
def evaluate_naive_bayes(conditionalProb, numPositiveInstances, numNegativeInstances):
    """Evaluates the accuracy of the Naive Bayes classifier on test data."""
    correct_predictions = 0
    total_predictions = 0

    for (classLabel, document) in test_data:
        prediction = predict_naive_bayes(document, conditionalProb, numPositiveInstances, numNegativeInstances)
        if prediction == classLabel:
            correct_predictions += 1
        total_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

In [12]:
evaluate_naive_bayes(conditionalProb, numPositiveInstances, numNegativeInstances)

0.7939698492462312

5. Modify the code and see what happens if we do not use Laplace smoothing